In [ ]:
# 031323
# YIKAI MAO

# circuit simulator with fidelity analysis

In [ ]:
%env PYTHONHASHSEED=0

# standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.tools.jupyter import *
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import *
#from ibm_quantum_widgets import * # does not work when not in ibm cloud platform
from qiskit.providers.aer import QasmSimulator, AerSimulator
from qiskit.transpiler.passes import RemoveBarriers

# other libraries
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
import mapomatic as mm
import numpy as np
import matplotlib.pyplot as plt

# loading your IBM Quantum account(s)
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()
provider.backends()

In [ ]:
def run_ideal_simulation(circuit, shots, seed):
    # assume perfect device with no noise

    simulator = Aer.get_backend('aer_simulator')
    temp_qc = transpile(circuit, simulator, optimization_level=0)
    job_temp = simulator.run(temp_qc, shots=shots, seed_simulator=seed)
    temp_results = job_temp.result()

    # expected output is plotted
    #plot = plot_histogram(temp_results.get_counts())
    #plot.patch.set_facecolor('xkcd:white')
    #display(plot)
    #print('raw outputs:')
    #print(temp_results.get_counts())

    # build simulation result dict
    num_clbits = circuit.num_clbits
    ideal_result_dict = {}

    for i in range(pow(2, num_clbits)):
        bin_str = format(i, str('0>' + str(num_clbits) + 'b'))
        if bin_str not in temp_results.get_counts():
            #print(bin_str + ': probability = 0')
            ideal_result_dict[bin_str] = 0
        else:
            ideal_result_dict[bin_str] = temp_results.get_counts()[bin_str]

    #print('simulation result dictionary:')
    #if num_clbits <= 6:
    #    print(ideal_result_dict)
    #else:
    #    print('result too long, skip')
        
    return ideal_result_dict

In [ ]:
def default_cost_return_fid(circ, layouts, backend):
    """The default mapomatic cost function that returns the total
    error rate over all the layouts for the gates in the given circuit
    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list of lists): List of specified layouts
        backend (IBMQBackend): An IBM Quantum backend instance
    Returns:
        list: Tuples of layout and error
    """
    out = []
    # Make a single layout nested
    props = backend.properties()
    for layout in layouts:
        error = 0
        fid = 1
        for item in circ._data:
            if item[0].name == 'cx':
                q0 = circ.find_bit(item[1][0]).index
                q1 = circ.find_bit(item[1][1]).index
                fid *= (1-props.gate_error('cx', [layout[q0],
                                                  layout[q1]]))

            elif item[0].name in ['sx', 'x']:
                q0 = circ.find_bit(item[1][0]).index
                fid *= 1-props.gate_error(item[0].name, layout[q0])

            elif item[0].name == 'measure':
                q0 = circ.find_bit(item[1][0]).index
                fid *= 1-props.readout_error(layout[q0])
        error = 1-fid
        #out.append((layout, error))
        out.append((layout, fid))
    return out

In [ ]:
# define circuit here, either a Qiskit circuit object or OpenQASM string
qasm_str = '''



'''

#circuit = QuantumCircuit.from_qasm_str(qasm_str)

In [ ]:
title = 'adder_n4'

In [ ]:
with open('circuits_small/'+title+'.qasm', 'r') as f:
    qasm_str = f.read()
circuit = QuantumCircuit.from_qasm_str(qasm_str)

In [ ]:
circuit.draw(scale = 0.5)

In [ ]:
from datetime import datetime

t = datetime(year=2023, month=1, day=1, hour=12)

backend = provider.get_backend('ibm_nairobi')
backend.properties(datetime=t).last_update_date

In [ ]:
from qiskit_aer.noise import NoiseModel

noise_model = NoiseModel.from_backend_properties(backend.properties(datetime=t))

backend_name = 'ibm_nairobi'

# generate a simulator that mimics the real quantum system with the latest calibration results
#backend_sim = AerSimulator.from_backend(backend)

# generate a simulator that mimics the real quantum system with previous calibrations
backend_sim = AerSimulator(noise_model=noise_model)

# use a fake device, from: https://qiskit.org/documentation/apidoc/providers_fake_provider.html
# 
# 27 qubit device = FakeToronto(), FakeSydney(), FakeMumbai(), FakeMontreal(), FakeKolkata(), FakeHanoi(), FakeCairo(), *FakeCambridge(), *FakeParis()
# 20 qubit device = FakeTokyo(), *FakeAlmaden(), *FakeBoeblingen(), *FakeJohannesburg(), *FakePoughkeepsie(), *FakeSingapore()
# 16 qubit device = FakeRueschlikon(), FakeGuadalupe()
# 14 qubit device = FakeMelbourne()
# 7 qubit device  = FakeNairobi(), FakeLagos(), FakeJakarta(), FakeCasablanca()
# 5 qubit device  = FakeYorktown(), FakeVigo(), FakeValencia(), FakeTenerife(), FakeRome(), FakeQuito(), FakeOurense(), FakeManila(), FakeLondon(),
#                   FakeLima(), FakeEssex(), FakeBurlington(), FakeBogota(), FakeBelem(), FakeAthens()
# ? qubit device  = *FakeBrooklyn(), *FakeManhattan(), *FakeRochester(), *FakeSantiago()
#               * = ibm didn't provide enough info for these devices
from qiskit.providers.fake_provider import *

#backend = FakeMontreal()
#backend_sim = AerSimulator.from_backend(backend)
#backend_name = 'fake_montreal'

In [ ]:
# do a test transpilition, just to see the number of CXs
# run this cell multiple times to see different transpile results
trans_qc = transpile(circuit, backend, optimization_level=0)
print('# of CXs =', trans_qc.count_ops()['cx'])
display(trans_qc.draw(scale = 0.5))

# show qubit layout on the device
print('physical qubit layout:')
display(plot_gate_map(backend))
print('virtual qubit layout:')
display(plot_circuit_layout(trans_qc, backend, view='virtual'))

In [ ]:
trans_qc._layout

In [ ]:
layouts = mm.matching_layouts(mm.deflate_circuit(trans_qc), backend)
mm_list = default_cost_return_fid(mm.deflate_circuit(trans_qc), layouts, backend)
sorted_list = sorted(mm_list, key=lambda mm: mm[1])
layout = sorted_list[-1][0]
sorted_list

In [ ]:
layouts = mm.matching_layouts(mm.deflate_circuit(trans_qc), backend)
mm_list = default_cost_return_fid(mm.deflate_circuit(trans_qc), layouts, backend)
sorted_list = sorted(mm_list, key=lambda mm: mm[1])
layout = sorted_list[-1][0]
sorted_list

In [ ]:
mm_fid = default_cost_return_fid(mm.deflate_circuit(trans_qc), [layout], backend)
mm_fid = mm_fid[0][1]
print(mm_fid)

In [ ]:
# some parameters for simulation

shots = 1024
seed = None

In [ ]:
# simulate this circuit
# assume perfect device with no noise

simulator = Aer.get_backend('aer_simulator')
job_temp = simulator.run(trans_qc, shots=shots, seed_simulator=seed)
temp_results = job_temp.result()

# expected output is plotted
plot = plot_histogram(temp_results.get_counts())
plot.patch.set_facecolor('xkcd:white')
display(plot)
print('raw outputs:')
print(temp_results.get_counts())

# build simulation result dict
num_clbits = trans_qc.num_clbits
sim_result_dict = {}

for i in range(pow(2, num_clbits)):
    bin_str = format(i, str('0>' + str(num_clbits) + 'b'))
    if bin_str not in temp_results.get_counts():
        #print(bin_str + ': probability = 0')
        sim_result_dict[bin_str] = 0
    else:
        sim_result_dict[bin_str] = temp_results.get_counts()[bin_str]

print('simulation result dictionary:')
if num_clbits <= 6:
    print(sim_result_dict)
else:
    print('result too long, skip')

In [ ]:
# run a noisy simulation
# calibration data maybe out of date

trans_qc = transpile(circuit, backend, initial_layout=layout, optimization_level=0)

job_temp = backend_sim.run(trans_qc, shots=shots, seed_simulator=seed)
temp_results = job_temp.result()
plot = plot_histogram(temp_results.get_counts())
plot.patch.set_facecolor('xkcd:white')
plot

In [ ]:
#with open('ibm_nairobi_on_111522_0230PM.pickle', 'wb') as f:
#    pickle.dump(backend, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('circuit_list_for_generator_single.pickle', 'wb') as f:
    pickle.dump(trans_qc, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
sim_result_dict = []
job_list = []
r2_list = []
num_clbits = trans_qc.num_clbits

num_tests = 50

for i in tqdm(list(range(num_tests))):
    sim_result_dict.append(run_ideal_simulation(trans_qc, shots, seed))
    job_list.append(backend_sim.run(trans_qc, shots=shots, seed_simulator=seed))
    
    temp_results = job_list[i].result()
    temp_result_dict = {}
        
    # build full result dict
    for j in range(pow(2, num_clbits)):
        bin_str = format(j, str('0>' + str(num_clbits) + 'b'))
        if bin_str not in temp_results.get_counts():
            #print(bin_str + ': probability = 0')
            temp_result_dict[bin_str] = 0
        else:
            temp_result_dict[bin_str] = temp_results.get_counts()[bin_str]
    
    r2_list.append(r2_score(list(sim_result_dict[i].values()), list(temp_result_dict.values())))
    
print(r2_list)

In [ ]:
title = title
q_fid = 0.

mm_fid = mm_fid
r2_list = r2_list

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
ax.grid(color="dimgray", linestyle='dotted', linewidth=0.5)
#ax.set(xlabel='job index', ylabel='circuit fidelity', title='noisy simulation')
ax.set(title=title)
ax.set(ylim=(0, 1), yticks=[0, 0.5, 1], xticks=[0, 25, 50])
ax.xaxis.set_tick_params(labelsize='small')
ax.yaxis.set_tick_params(labelsize='small')

x = range(len(r2_list))
avg = np.mean(r2_list)
ma = np.max(r2_list)
mi = np.min(r2_list)

ax.plot(x, r2_list, color='dimgray', linewidth=3, label='simulated fidelity')
#ax.axhline(avg, color='red', linewidth=1, label='mean fidelity')
#ax.text(0, avg, "{:.2f}".format(avg), color="red", ha="right", va="top", fontsize='xx-large')
#ax.annotate("mean: {:.2f}".format(avg), xy=(0,avg-0.15), color='red', fontsize='x-large')
ax.axhline(q_fid, color='purple', linewidth=2, label='q-fid estimation')
ax.annotate("q-fid: {:.2f}".format(q_fid), xy=(50,q_fid+0.1), color='purple', fontsize='x-large', ha="right", va="top")
ax.text(1.25, 0.1, # 0.7 or 0.1
        "max:   {:.2f}".format(ma)+'\n'+"mean: {:.2f}".format(avg)+'\n'+"min:    {:.2f}".format(mi),
        fontsize='medium',
        color="dimgray",
        bbox=dict(facecolor='none', edgecolor='dimgray', boxstyle='round,pad=0.3'))
#ax.legend(loc=0,fontsize='large')

fig.savefig(title+'_'+backend_name+'.png')

ax.axhline(mm_fid, color='dimgray', linewidth=1, label='mm estimation')
ax.annotate("mm: {:.2f}".format(mm_fid), xy=(50,mm_fid-0.02), color='dimgray', fontsize='medium', ha="right", va="top")

fig.savefig(title+'_'+backend_name+'_with_mm.png')

In [ ]:
from sklearn.metrics import mean_squared_error
testq = [q_fid]*50
testmm = [mm_fid]*50
print('MSE q_fid:', mean_squared_error(r2_list, testq))
print('MSE mm_fid:', mean_squared_error(r2_list, testmm))